In [ ]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_tx_task import SingleChTxTask
from rfsoc_rfdc.receiver.single_ch_rx_task import SingleChRxTask

from rfsoc_rfdc.transmitter.multi_ch_tx_task import MultiChTxTask
from rfsoc_rfdc.receiver.multi_ch_rx_task import MultiChRxTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

In [ ]:
# RFSoC experiment configuration
# +--------------------------------------------------------------------------------------+
# | Feature: Single channel 100MHz bandwidth (Tx only)                                   |
# +-----------------+--------------------+----------------+-----------+-----+------------+
# | DAC Sample Rate | Interpolation Rate | IQ Sample Rate | Bandwidth | NCO | Ref. Clock |
# | 2.0 GSPS        | 20x                | 200 MSPS       | 100M      | +1G | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-----+------------+
# | ADC Sample Rate | Decimation Rate    | IQ Sample Rate | Bandwidth | NCO | Ref. Clock |
# | 2.0 GSPS        | 20x                | 200 MSPS       | 100M      | -1G | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-----+------------+

In [ ]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v27.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 2000.0,
    "DACInterpolationRate": 20,
    "DACNCO": 1000.0,
    "ADCSampleRate": 2000.0,
    "ADCInterpolationRate": 20,
    "ADCNCO": -1000.0
}
ZCU216_CONFIG.update(NEW_CONFIG)

In [ ]:
import sys
import os

rfdc_t = RfdcTask(ol)
led_t = BlinkLedTask(ol)
tx_t = SingleChTxTask(ol, "./wave_files/Tx.mat")
rx_t = SingleChRxTask(ol)

for task in [rfdc_t]:
    task.start()
    task.join()

parallel_task = [led_t, tx_t]
for task in parallel_task:
    task.start()
for task in parallel_task:
    task.join()